In [1]:
import glob
from pathlib import Path
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Utils

# Job requirement

## Analyze requirement keywords

In [26]:
print(df.requirement_list_cleaned.shape)
df.requirement_list_cleaned.apply(len).sum()


(90997,)


553875

In [50]:
import itertools
from collections import Counter

In [ ]:
query_column = 'blue_or_white'
query_word = 'blue'
querydf = df.loc[df[query_column] == query_word]

keywords = list(itertools.chain(*querydf.loc[:,'tokenized_requirement']))
counts = Counter(keywords)

In [ ]:
query_words_degree = ['trung cấp', 'trung học',  
'không yêu cầu',
                    'chứng chỉ', 'trung học phổ thông','trung cấp - nghề', 
                    'chứng chỉ chuyên ngành', 'lao động phổ thông', 'chứng chỉ - sơ cấp nghề',
                    ]

In [ ]:
keywords = {
    'kn1-giaotiep': ['giao tiếp', 'đàm phán', 'thuyết phục', 'thuyết trình', 'thân thiện', 'rõ ràng',
                    'trình bày', 'tác phong', 'lắng nghe', 'truyền đạt', 'cởi mở', 'khéo léo',
                    'tương tác', 'tiếp thu', 'phản biện', 'hoạt ngôn', 'hòa nhã', 'thương thảo',
                    'ứng xử', 'thương thuyết', 'tự nhiên', 'trôi chảy', 'rành mạch',
                    'linh hoạt', 'hợp tác', 'ứng biến'],
    'kn2-xahoi': ['nhóm', 'đàm phán', 'thuyết phục', 'thuyết trình', 'chăm sóc', 'khách hàng',
                    'lãnh đạo', 'trình bày', 'thương lượng', 'phản biện'     ],
    'kn3': [],
    'kn4': [],
    'kn5': [],
    'kn6': [],
    'kn7': []
}

In [101]:
keywords2 = {
'giaotiep': ['giao_tiếp',  'thân_thiện', 'rõ_ràng', 'trình_bày','tác_phong','lắng_nghe', 'truyền_đạt', 'cởi_mở', 'khéo_léo', 
                'tương_tác', 'tiếp_thu', 'phản_biện', 'hoạt_ngôn','hòa_nhã', 'giao_tiếp_ứng_xử',  'tự_nhiên','trôi_chảy','rành_mạch', 
                'giao_tiếp_linh_hoạt', 'giao_tiếp_hợp_tác', 'giao_tiếp_ứng_biến'
                    ],
'xahoi': ['làm_việc nhóm', 'đàm_phán', 'thuyết_phục', 'thuyết_trình', 'chăm_sóc','thương_thảo', 'thương_thuyết',
                    'khách_hàng', 'lãnh_đạo', 'trình_bày', 'thương_lượng', 'phản_biện', 'quản_lý', 'thời_gian', 
                    'quản_lý thời_gian', ],
        
'phantich': ['phân_tích', 'giải_quyết vấn_đề', 'nhạy_bén', 'tư_duy logic', 'quyết_đoán', 'xử_lý vấn_đề', 
                'problem solving', 'ứng_biến', 'lập_luận', 'xử_lý_giải_quyết', 'nhìn_nhận', 'vấn_đề', 'đào_sâu'],

'congnghe': ['excel', 'tin_học văn_phòng', 'word', 'vi_tính', 'vi_tính văn_phòng', 'ms office', 'microsoft office', 
        'microsoft word', 'microsoft excel'],


'thaido': ['trung_thực', 'năng_động', 'nhanh_nhẹn', 'nhiệt_tình', 'cẩn_thận', 'tinh_thần trách_nhiệm', 
            'chủ_động', 'linh_hoạt', 'thái_độ', 'tự_tin', 'hòa_đồng', 
            'tác_phong chuyên_nghiệp'],


'kn6': ['độc_lập','kỹ_năng lập_kế_hoạch', 'áp_lực'],

'ngoaingu': ['ngoại_ngữ','tiếng_anh giao_tiếp','tiếng anh giao_tiếp','tiếng anh tốt',
                'tiếng anh cơ_bản','tiếng anh thành_thạo','giao_tiếp tiếng anh cơ_bản']
}

In [84]:
def remove_underscore(text):    
    # remove html markup
    text=re.sub("_"," ",text)
    return text

removeUnderscore = lambda x: remove_underscore(x)

In [110]:
def count_keyword_frequency_vi(df,query_column, query_value, path, save=False, ):
    counts_df = pd.DataFrame()
    for key in list(keywords2.keys()):
        keywords = keywords2[key]
        querydf = df.loc[df[query_column] == query_value]
        querydf = querydf.loc[~df['tokenized_requirement_2'].isnull()]
        
        words = list(itertools.chain(*querydf.loc[:,'tokenized_requirement_2']))
        counts = Counter(words)

        counts_df_ = pd.DataFrame.from_dict(counts, orient='index').reset_index()
        counts_df_.columns = [f'{key}','count']

        counts_df_ = counts_df_.loc[counts_df_[f'{key}'].isin(keywords)]
        counts_df_ = counts_df_.sort_values('count',ascending=False).reset_index(drop=True)

        counts_df_[f'{key}'] = counts_df_[f'{key}'].apply(removeUnderscore)

        counts_df = pd.concat([counts_df,counts_df_],axis=1)
        # counts_df.to_excel(f"counters/{query_column} - {query_value}.xlsx", engine='xlsxwriter')

        if save == True:
            # writer = pd.ExcelWriter(path, engine='openpyxl', if_sheet_exists='replace', mode='a')
            writer = pd.ExcelWriter(path, engine='openpyxl', )
            counts_df.to_excel(writer, sheet_name = f"{query_column}-{query_value}",index=False)
            writer.save()
            writer.close()

        
    return counts_df

In [ ]:
for d in query_words_degree:
    count_keyword_frequency_vi(df,'degree',d,path = 'counters/vi.xlsx', save=True)

In [116]:
count_keyword_frequency_vi(dfvi,'blue_or_white','white',path = 'counters/vi.xlsx', save=True)

,giaotiep,count,xahoi,count,phantich,count,congnghe,count,thaido,count,kn6,count,ngoaingu,count
0,giao tiếp,29304,quản lý,18673.0,phân tích,9305.0,excel,8517.0,trung thực,10660.0,áp lực,13879.0,ngoại ngữ,3741.0
1,thân thiện,2192,khách hàng,11182.0,vấn đề,7040.0,word,5964.0,năng động,8906.0,độc lập,9233.0,NaN,NaN
2,rõ ràng,2088,thời gian,6370.0,nhạy bén,2101.0,vi tính,5252.0,nhanh nhẹn,8845.0,NaN,NaN,NaN,NaN
3,trình bày,2026,đàm phán,5740.0,quyết đoán,741.0,NaN,NaN,nhiệt tình,7454.0,NaN,NaN,NaN,NaN
4,tác phong,1552,thuyết phục,4889.0,nhìn nhận,159.0,NaN,NaN,cẩn thận,7266.0,NaN,NaN,NaN,NaN
5,lắng nghe,1199,chăm sóc,3765.0,ứng biến,81.0,NaN,NaN,chủ động,6274.0,NaN,NaN,NaN,NaN
6,truyền đạt,1019,thuyết trình,3447.0,lập luận,38.0,NaN,NaN,linh hoạt,3002.0,NaN,NaN,NaN,NaN
7,khéo léo,815,lãnh đạo,2057.0,xử lý giải quyết,24.0,NaN,NaN,thái độ,2790.0,NaN,NaN,NaN,NaN
8,cởi mở,725,trình bày,2026.0,đào sâu,5.0,NaN,NaN,tự tin,2098.0,NaN,NaN,NaN,NaN
9,tiếp thu,645,thương lượng,1403.0,NaN,NaN,NaN,NaN,hòa đồng,969.0,NaN,NaN,NaN,NaN


In [151]:
df.loc[(df.language =='en') & (df.blue_or_white == 'white')].shape
df.loc[(df.language =='en') ].shape

(12474, 19)

In [ ]:
def cleaner(text):
    texts = text.split()
    l = []
    for t in texts:
        l.append(re.sub('_',' ',t))
    return l

def cleaner_2(text):
    texts = text.split()
    return texts

In [154]:
skills = {
    'communication': ['commun', 'written','verbal','present','write','oral','speak','spoken','japanes','korean','chines'],
    'social': ['interperson','leadership','teamwork','coordin','custom','negoti','manag'],
    'problem-solving': ['solv','analyt','creativ'],
    'computer': ['microsoft' ,'offic', 'ms offic','comput'],
    'attitude': ['independ','pressur','attitud'],
    'others': ['detail','english', ]    
}

In [144]:
def count_keyword_frequency_en(df,query_column, query_value, path, save=False, ):
    counts_df = pd.DataFrame()
    for key in list(skills.keys()):
        keywords = skills[key]
        querydf = df.loc[df[query_column] == query_value]
        querydf = querydf.loc[~df['tokenized_requirement'].isnull()]
        
        words = list(itertools.chain(*querydf.loc[:,'tokenized_requirement']))
        counts = Counter(words)

        counts_df_ = pd.DataFrame.from_dict(counts, orient='index').reset_index()
        counts_df_.columns = [f'{key}','count']

        counts_df_ = counts_df_.loc[counts_df_[f'{key}'].isin(keywords)]
        counts_df_ = counts_df_.sort_values('count',ascending=False).reset_index(drop=True)

        counts_df_[f'{key}'] = counts_df_[f'{key}'].apply(removeUnderscore)

        counts_df = pd.concat([counts_df,counts_df_],axis=1)
        # counts_df.to_excel(f"counters/{query_column} - {query_value}.xlsx", engine='xlsxwriter')

        if save == True:
            # writer = pd.ExcelWriter(path, engine='openpyxl', if_sheet_exists='replace', mode='a')
            writer = pd.ExcelWriter(path, engine='openpyxl', )
            counts_df.to_excel(writer, sheet_name = f"{query_column}-{query_value}",index=False)
            writer.save()
            writer.close()

        
    return counts_df

In [155]:
count_keyword_frequency_en(df,'language','en', path ='en.xlsx', save=True)

,communication,count,social,count,problem-solving,count,computer,count,attitude,count,others,count
0,commun,9363,manag,8789.0,solv,2683.0,offic,3073.0,pressur,1860.0,english,9723.0
1,written,2827,custom,2507.0,analyt,2355.0,comput,2827.0,independ,1810.0,detail,1836.0
2,write,2225,interperson,1767.0,creativ,1236.0,microsoft,1726.0,attitud,1145.0,NaN,NaN
3,present,1635,negoti,1440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,verbal,1506,leadership,1175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,speak,1249,teamwork,1111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,spoken,950,coordin,312.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,japanes,840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,oral,595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,chines,371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Statistics

In [49]:
df.source.value_counts()
# df.loc[df.language=='en'].source.value_counts()
# df.shape

careerBuilder     23922
topCV             23603
jobsGo            18397
sieuthivieclam    13964
vietnamworks       7947
vieclam24h         1102
tuyencongnhan      1082
timviecnhanh        980
Name: source, dtype: int64

In [29]:
dfvi = df.loc[df.language=='vi'].reset_index(drop=True)
print(dfvi.shape)
print(dfvi.requirement_list_cleaned.apply(len).sum())

(78523, 19)
434893


In [48]:
dfen = df.loc[df.language=='en'].reset_index(drop=True)
print(dfen.shape)
print(dfen.requirement_list_cleaned.apply(len).sum())

(12474, 19)
118982


In [32]:
434893 + 118982

553875

## giao_tiep - commnunication both `en` and `vi`

In [44]:
dfen

,url,title,salary,experiment,degree,so_luong,industry,location,level,jd,benefit,requirement,source,language,blue_or_white,requirement_list,requirement_list_cleaned,tokenized_requirement,tokenized_requirement_2
0,https://vieclam24h.vn/ke-toan-kiem-toan/ar-acc...,Ar Accountant - Intern,3 - 5 triệu,Chưa có kinh nghiệm,đại học,1.0,Kế toán - Kiểm toánLàm bán thời gianThực tập,TP.HCM,Chuyên viên- nhân viên,- Be responsible for day to day finance and ac...,- Monthly salary- Good environment for enrich ...,- Either a last year student or a fresh gradua...,vieclam24h,en,white,[Either a last year student or a fresh graduat...,"[either last year student fresh graduat, well,...",NaN,NaN
1,https://vieclam24h.vn/det-may-da-giay/nhan-vie...,Nhân Viên Mua Hàng (Purchansing Staff),10 - 15 triệu,1 năm,cao đẳng,10.0,Dệt may - Da giày,Hà NộiBắc NinhBắc Giang,Chuyên viên- nhân viên,- Understand the specifications & familiar wit...,- Annual bonus- Annual health check-up- 14 day...,- Have working experience in purchasing area 1...,vieclam24h,en,white,[Have working experience in purchasing area 1 ...,[work experi purchas area year kinh nghiệm mua...,NaN,NaN
2,https://vieclam24h.vn/it-phan-cung-mang/it-inf...,IT Infrastructure Internship,7 - 10 triệu,1 năm,cao đẳng,2.0,IT phần cứng/mạngIT phần mềmThực tập,TP.HCM,Cộng tác viên,"-\tKnowledge of Desktop, Servers & Network [LA...",- Mức lương từ 6 đến 8 triệu.- Làm việc từ thứ...,"-\tKnowledge of Desktop, Servers & Network [LA...",vieclam24h,en,white,"[Knowledge of Desktop, Servers & Network [LAN ...",[knowledg desktop server network lan wan switc...,NaN,NaN
3,https://vieclam24h.vn/ngoai-thuong-xuat-nhap-k...,Customer Service Executive,10 - 15 triệu,1 năm,cao đẳng,2.0,Dịch vụNgoại thương - Xuất nhập khẩuTư vấn,TP.HCM,Chuyên viên- nhân viên,- Monitoring shipping documents and customer s...,- Training Opportunities: Advance level of ski...,- From 22 years old- University or college gra...,vieclam24h,en,white,"[From 22 years old, University or college grad...","[year old, univers colleg graduat relat econom...",NaN,NaN
4,https://vieclam24h.vn/det-may-da-giay/producti...,Production Team Leader,10 - 15 triệu,3 năm,cao đẳng,5.0,Cơ khí - Chế tạoDệt may - Da giàyĐiện - Điện t...,TP.HCMLong AnBình Dương,Quản lý nhóm- giám sát,"- Shift handover meeting, morning shift and ev...",- Yearly performance review.- 13th month payme...,"- Vocational, College or Bachelor degree relat...",vieclam24h,en,white,"[Vocational, College or Bachelor degree relate...","[vocat colleg bachelor degre relat technic, ye...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12469,https://sieuthivieclam.vn/viec-lam/receptionis...,RECEPTIONIST - RESTAURANHT WAITRESS(Mã:CV2016-...,\nThoả thuận\n,\nKhông yêu cầu\n,lao động phổ thông,\n10\n,\nDu lịch nhà hàng - Khách sạn - Ăn uống\n,\nNơi làm việc:\nQuận 10\n,\nNhân viên\n,\nRecruitment information:\r\n- Receptionist\r...,\nPhúc lợi\nThỏa thuận\n,\nRequirements:\r\n- Good appearance\r\n- Engl...,sieuthivieclam,en,blue,"[Requirements:, Good appearance, English commu...","[requir, good appear, english commun, custom o...",NaN,NaN
12470,https://sieuthivieclam.vn/viec-lam/nhan-vien-d...,NHÂN VIÊN ĐỒ HỌA(Mã:CV2016-A2435),\nThoả thuận\n,\nKhông yêu cầu\n,lao động phổ thông,\n1\n,\nThiết kế - Xây dựng\n,\nNơi làm việc:\nQuận 1\n,\nNhân viên\n,\n\r\n- Thử việc 1 tháng: 5.000.000 d0/tháng+p...,\nPhúc lợi\nThỏa thuận\n,"\n\r\n- Rành Corel, photoshop\r\n- Illustrante...",sieuthivieclam,en,blue,"[Rành Corel, photoshop, Illustranter, fast...]","[rành corel photoshop, illustrant fast]",NaN,NaN
12471,https://sieuthivieclam.vn/viec-lam/customer-se...,Customer Service Night Shift(Mã:CV2016-A0493),\n7.000.000VNĐ\n,\n1\n,trung cấp,\n2\n,\nChăm sóc khách hàng - Điện thoại viên - Quản...,\nNơi làm việc:\nQuận Tân Bình\n,\nNhân viên\n,\n- Conduct oversea call to consult products f...,"\nPhụ cấp\n-10,000,000\nPhúc lợi\nThỏa thuận\n",\n- Good at communication and negotiation skil..

In [54]:
# query_column = 'language'
# query_word = 'vi'
# querydf = df.loc[df[query_column] == query_word]
# keywords = list(itertools.chain(*dfvi.loc[:,'tokenized_requirement_2']))
keywords = list(itertools.chain(*dfen.loc[:,'tokenized_requirement']))
counts = Counter(keywords)

counts_df_ = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts_df_.columns = ['keywords','count']

# keywords = ['giao_tiếp']
keywords = ['commun','communication','communi']
counts_df_ = counts_df_.loc[counts_df_['keywords'].isin(keywords)]
counts_df_ = counts_df_.sort_values('count',ascending=False).reset_index(drop=True)
counts_df_

,keywords,count
0,commun,9363
1,communication,2


In [61]:
df.language.value_counts()
# df.shape

vi    78523
en    12474
Name: language, dtype: int64

In [152]:
import plotly.express as px
data = dict(
    character=["Total 90997 jobs",  "78523 Vietnamese", "12474 English", "33226 Giao tiếp","9365 Communication" ],
    parent=["", "Total 90997 jobs", "Total 90997 jobs",  "78523 Vietnamese" ,"12474 English"],
    value=[90997, 78523,12474, 33226,9365, ])

fig =px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    branchvalues="total",
)
fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=1, l=30, r=30, b=1),
    width=600, height=600
)
fig.show()

In [69]:
import plotly.express as px
data = dict(
    character=["Total 553875", "118982 English", "434893 Vietnamese", "Communication","Giao tiếp" ],
    parent=["", "Total 553875", "Total 553875", "118982 English", "434893 Vietnamese" ],
    value=[553875, 118982, 434893, 9365, 33226,])

fig =px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    branchvalues="total",
)
fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=1, l=30, r=30, b=1),
    width=300, height=300
)
fig.show()

### Vietnamese keywords in giao_tiep

In [151]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],)

ids = ['total','vi','en'] + list('vi - '+df.source.unique()) + list('en - '+df.source.unique()) 
labels = [f'Total {df.shape[0]} jobs','vi','en'] + list(''+df.source.unique()) +list(''+df.source.unique()) 
parents = ['','total','total'] + ['vi']*8 + ['en']*8

values = [df.shape[0]] + [df.loc[df.language=='vi'].shape[0]] + [df.loc[df.language=='en'].shape[0]]

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'vi')].shape[0])

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'en')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 1)
ids = ['total' ]+ list(df.source.unique()) + list(df.source.unique()+' - en') + list(df.source.unique()+' - vi') 
labels = [f'Total {df.shape[0]} jobs' ]+ list(df.source.unique()) + ['en']*8 + ['vi']*8
parents = [""] + ["total"]*8 + list(df.source.unique()) *2

values = [df.shape[0]]
for source in df.source.unique():
    values.append(df.loc[df.source==source].shape[0])
for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.language == 'en')].shape[0])
for source in df.source.unique():  
    values.append(df.loc[(df.source==source) & (df.language == 'vi')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 2)



fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=1, l=30, r=30, b=1),
    width=1200, height=600
)

fig.show()

In [157]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],
                     )

ids = ['total','blue','white'] + list('blue - '+df.source.unique()) + list('white - '+df.source.unique()) 
labels = [f'Total {df.shape[0]} jobs','thợ','thầy'] + list(''+df.source.unique()) +list(''+df.source.unique()) 
parents = ['','total','total'] + ['blue']*8 + ['white']*8

values = [df.shape[0]] + [df.loc[df.blue_or_white=='blue'].shape[0]] + [df.loc[df.blue_or_white=='white'].shape[0]]

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'blue')].shape[0])

for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'white')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 1)
ids = ['total' ]+ list(df.source.unique()) + list(df.source.unique()+' - en') + list(df.source.unique()+' - vi') 
labels = [f'Total {df.shape[0]} jobs' ]+ list(df.source.unique()) + ['thợ']*8 + ['thầy']*8
parents = [""] + ["total"]*8 + list(df.source.unique()) *2

values = [df.shape[0]]
for source in df.source.unique():
    values.append(df.loc[df.source==source].shape[0])
for source in df.source.unique():
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'blue')].shape[0])
for source in df.source.unique():  
    values.append(df.loc[(df.source==source) & (df.blue_or_white == 'white')].shape[0])

fig.add_trace(go.Sunburst(ids = ids,
                            labels = labels,
                            parents=parents,
                            values = values,
                             branchvalues="total",
                            ), 1, 2)



fig.update_layout(
    grid= dict(columns=2, rows=1),
    margin = dict(t=1, l=30, r=30, b=1),
    width=1200, height=600,
)
# fig.set_tit
fig.show()

In [96]:
import datetime
date_object = datetime.date.today()
df.to_csv(f'aggregated-{date_object}.csv', index=False)

# Industry

In [16]:
import re
def string_to_list(text):
    string_list = re.split('\n|\t|•|-|,',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [17]:
df[df['industry'].isnull()]#['source'].value_counts()#.reset_index(drop=True).astype(str)


,url,title,salary,experiment,degree,so_luong,industry,location,level,jd,benefit,requirement,source,language,blue_or_white,requirement_list,requirement_list_cleaned,tokenized_requirement,tokenized_requirement_2


In [18]:
df.loc[df.source == 'vietnamworks', 'industry'] = df.loc[df.source == 'vietnamworks', 'industry'].apply(stringToList)

In [19]:
df.source.unique()

array(['vieclam24h', 'jobsGo', 'tuyencongnhan', 'topCV', 'careerBuilder',
       'timviecnhanh', 'vietnamworks', 'sieuthivieclam'], dtype=object)

In [20]:

def string_to_list(text):
    text=re.sub("IT","It",text)
    text=re.sub("PR","Pr",text)
    text=re.sub("-","",text)
    text=" ".join(text.split())
    string_list = re.findall('[A-Z][^A-Z]*', text)
    # string_list = [re.split('-',t) for t in string_list]

    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [21]:
df.loc[df.source == 'vieclam24h', 'industry'] = df.loc[df.source == 'vieclam24h', 'industry'].apply(stringToList)

In [22]:
def string_to_list(text):
    string_list = re.split('\n|\t|•|-|,',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))[:-1]

stringToList = lambda x: string_to_list(x)

In [23]:
df.loc[df.source == 'jobsGo', 'industry'] = df.loc[df.source == 'jobsGo', 'industry'].apply(stringToList)

In [24]:
def string_to_list(text):
    string_list = re.split('\n',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [25]:
df.loc[df.source == 'tuyencongnhan', 'industry'] = df.loc[df.source == 'tuyencongnhan', 'industry'].apply(stringToList)

In [26]:
def string_to_list(text):
    string_list = re.split('\n|/',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [27]:
df.loc[df.source == 'topCV', 'industry'] = df.loc[df.source == 'topCV', 'industry'].apply(stringToList)

In [28]:
def string_to_list(text):
    string_list = re.split(',|/',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [29]:
df.loc[df.source == 'careerBuilder', 'industry']  = df.loc[df.source == 'careerBuilder', 'industry'].apply(stringToList)

In [30]:
def string_to_list(text):
    # string_list = re.split(',|/',text)
    # string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    text=re.sub("/","",text)
    text=re.sub(" - ","",text)
    string_list = re.findall('[A-Z][^A-Z]*', text)
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [31]:
df.loc[df.source == 'timviecnhanh', 'industry']  = df.loc[df.source == 'timviecnhanh', 'industry'].apply(stringToList)

In [32]:
def string_to_list(text):
    string_list = re.split('\n|-',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [33]:
df.loc[df.source == 'sieuthivieclam', 'industry'] = df.loc[df.source == 'sieuthivieclam', 'industry'].apply(stringToList)

In [145]:
# querydf = df.loc[df.blue_or_white == 'white']
querydf = df

keywords = list(itertools.chain(*querydf.loc[:,'industry']))
counts = Counter(keywords)

counts_df_ = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts_df_.columns = ['word','count']

counts_df_ = counts_df_.sort_values('count',ascending=False).reset_index(drop=True)

In [146]:
def string_to_list(text):
    string_list = re.split('/|,|:',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [147]:
list(itertools.chain(*counts_df_.loc[:,'word']))
counts_df_.word = counts_df_.word.apply(stringToList)

In [148]:
counts_df_['words'] = counts_df_['word']*counts_df_['count']
counts_df_

,word,count,words
0,[Bán hàng],18096,"[Bán hàng, Bán hàng, Bán hàng, Bán hàng, Bán h..."
1,[Kinh doanh],16964,"[Kinh doanh, Kinh doanh, Kinh doanh, Kinh doan..."
2,[Marketing],8254,"[Marketing, Marketing, Marketing, Marketing, M..."
3,[Tiếp thị],6602,"[Tiếp thị, Tiếp thị, Tiếp thị, Tiếp thị, Tiếp ..."
4,[Kiểm toán],6511,"[Kiểm toán, Kiểm toán, Kiểm toán, Kiểm toán, K..."
...,...,...,...
761,[Dịch vụĐiện Điện tử Điện lạnh],1,[Dịch vụĐiện Điện tử Điện lạnh]
762,"[Civil, Construction]",1,"[Civil, Construction]"
763,"[Giáo Dục, Đào Tạo]",1,"[Giáo Dục, Đào Tạo]"
764,"[Bán hàng, Quản Trị, Nhân Sự]",1,"[Bán hàng, Quản Trị, Nhân Sự]"


In [149]:
keywords = list(itertools.chain(*counts_df_.loc[:,'words']))
counts = Counter(keywords)

counts_df_2 = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts_df_2.columns = ['word','count']

counts_df_2 = counts_df_2.sort_values('count',ascending=False).reset_index(drop=True)

In [150]:
counts_df_2

,word,count
0,Bán hàng,18179
1,Kinh doanh,16998
2,Marketing,8329
3,Kế toán,6659
4,Tiếp thị,6634
...,...,...
670,Trang thiết bị gia dụngĐiệnĐiện tửĐiện lạnh,1
671,Nội thấtĐiệnĐiện tửĐiện lạnh,1
672,FMCG,1
673,HVAC,1


In [151]:
df.industry[0]
counts_df_2['count'].sum()

319872

In [144]:
counts_df_2.iloc[:52,:]

,word,count
0,Bán hàng,14991
1,Kinh doanh,13891
2,Marketing,8285
3,Tài chính,6288
4,Quảng cáo,6099
5,Kế toán,5810
6,Kiểm toán,5663
7,Dịch vụ khách hàng,5429
8,Ngân hàng,5010
9,Hành chính,4961


In [34]:
df

,url,title,salary,experiment,degree,so_luong,industry,location,level,jd,benefit,requirement,source,language,blue_or_white,requirement_list,requirement_list_cleaned,tokenized_requirement,tokenized_requirement_2
0,https://vieclam24h.vn/nhan-vien-kinh-doanh/nha...,Nhân Viên Kinh Doanh Lĩnh Vực Chuyển Phát Nhan...,10 - 40 triệu,Chưa có kinh nghiệm,cao đẳng,5.0,"[Quản trị kinh doanh, Ngoại thương , Xuất nhập...",Hà Nội,Chuyên viên- nhân viên,"- Chủ động tìm kiếm thông tin khách hàng mới, ...",- Lương cứng 5 triệu + hoa hồng => Thu nhập từ...,- Ưu tiên có kinh nghiệm làm trong lĩnh vực gi...,vieclam24h,vi,white,[Ưu tiên có kinh nghiệm làm trong lĩnh vực gia...,[ưu tiên có kinh nghiệm làm trong lĩnh vực gia...,"[ưu tiên, có, kinh nghiệm, làm, trong, lĩnh vự...","[ưu_tiên, có, kinh_nghiệm, làm, trong, lĩnh_vự..."
1,https://vieclam24h.vn/ban-hang/nhan-vien-ban-h...,Nhân Viên Bán Hàng Tại Showroom Thu Nhập Trên ...,15 - 20 triệu,Dưới 1 năm,trung cấp,3.0,"[Bán hàng, Nhân viên kinh doanh, Tư vấn]",Hà Nội,Chuyên viên- nhân viên,"- Làm việc tại Showroom BigC The Garden, Mễ...",- Thu nhập: Từ 15-30 Triệu / Tháng- Lương thử ...,"- Có kinh nghiệm về bán hàng, yêu thích bán hà...",vieclam24h,vi,blue,"[Có kinh nghiệm về bán hàng, yêu thích bán hàn...",[có kinh nghiệm về bán hàng yêu thích bán hàng...,"[có, kinh nghiệm, về, bán, hàng, yêu thích, bá...","[có, kinh_nghiệm, về, bán, hàng, yêu_thích, bá..."
2,https://vieclam24h.vn/dich-vu/shopee-nhan-vien...,Shopee - Nhân Viên Chăm Sóc Khách Hàng,5 - 7 triệu,Dưới 1 năm,trung học,15.0,"[Dịch vụ, Thương mại điện tử, Tư vấn]",TP.HCM,Chuyên viên- nhân viên,MÔ TẢ CÔNG VIỆC- Tiếp nhận và giải đáp thắc mắ...,- Thu nhập: Lương cứng: 6.300.000VND/tháng + t...,- Tốt nghiệp THPT trở lên.- Tuổi từ 18 đến 32....,vieclam24h,vi,blue,"[Tốt nghiệp THPT trở lên., Tuổi từ 18 đến 32.,...","[tốt nghiệp thpt trở lên, tuổi từ đến, tối thi...","[tốt nghiệp, thpt, trở, lên, tuổi, từ, đến, tố...","[tốt_nghiệp, thpt, trở, lên, tuổi, từ, đến, tố..."
3,https://vieclam24h.vn/ban-hang/nhan-vien-tu-va...,Nhân Viên Tư Vấn Bán Hàng Hà Nội,7 - 10 triệu,1 năm,trung cấp,10.0,"[Bán hàng, Nhân viên kinh doanh, Thời trang]",Hà Nội,Chuyên viên- nhân viên,"- Tư vấn, giới thiệu và bán sản phẩm Vàng, Bạc...",- Thu nhập từ 6 - 9 triệu / tháng;- Hỗ trợ tiề...,"- Nam / Nữ, sức khỏe tốt, ngoại hình cân đối, ...",vieclam24h,vi,blue,"[Nam / Nữ, sức khỏe tốt, ngoại hình cân đối, ư...",[nam nữ sức khỏe tốt ngoại hình cân đối ưa nhì...,"[nam nữ, sức khỏe, tốt, ngoại hình, cân đối, ư...","[nam_nữ, sức_khỏe, tốt, ngoại_hình, cân_đối, ư..."
4,https://vieclam24h.vn/khach-san-nha-hang/nhan-...,"Nhân Viên Lễ Tân Tòa Nhà Chung Cư - Q.4, Q.Bìn...",7 - 10 triệu,Dưới 1 năm,trung cấp,8.0,"[Khách sạn , Nhà hàng, Hành chính , Văn phòng,...",TP.HCMLong An,Chuyên viên- nhân viên,"1. Công tác lễ tân:- Đón tiếp khách, trực điện...",1. Thu nhập: Xứng đáng theo năng lực thực tế2....,"- Tốt nghiệp Trung cấp, Cao đẳng trở lên;- Có ...",vieclam24h,vi,blue,"[Tốt nghiệp Trung cấp, Cao đẳng trở lên;, Có k...","[tốt nghiệp trung cấp cao đẳng trở lên, có kin...","[tốt nghiệp, trung cấp, cao đẳng, trở, lên, có...","[tốt_nghiệp, trung_cấp, cao_đẳng, trở, lên, có..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90992,https://sieuthivieclam.vn/viec-lam/nhan-vien-k...,NHÂN VIÊN KINH DOANH(Mã:CV2015-0946),\n4.000.000VNĐ\n,\nKhông yêu cầu\n,cao đẳng,\n10\n,"[Kinh doanh, Tiếp thị, Bán hàng]",\nNơi làm việc:\nQuận Tân Bình\n,\nNhân viên\n,\nNhân viên kinh doanh làm việc trong ngành đi...,\nPhụ cấp\nhoa hồng\nPhúc lợi\nThỏa thuận\n,\nTốt nghiệp Cao Đẳng trở lên. Thành thạo vi t...,sieuthivieclam,vi,white,[Tốt nghiệp Cao Đẳng trở lên. Thành thạo vi tí...,[tốt nghiệp cao đẳng trở lên thành thạo vi tín...,"[tốt nghiệp, cao đẳng, trở lên, thành thạo, vi...","[tốt_nghiệp, cao_đẳng, trở_lên, thành_thạo, vi..."
90993,https://sieuthivieclam.vn/viec-lam/giup-viec-n...,GIÚP VIỆC NHÀ( BAO ĂN Ở)(Mã:CV2015-0945),\n3.500.000VNĐ\n,\nKhông yêu cầu\n,lao động phổ thôn

In [73]:
df_tmp = df[df['industry'].str.contains('Cơ khí', regex=False)]
df_tmp.shape


(3916, 19)

In [74]:
df_tmp['blue_or_white'].value_counts()

white    2890
blue     1026
Name: blue_or_white, dtype: int64

In [87]:
# df_tmp.loc[df_tmp['blue_or_white']=='blue', 'tokenized_requirement_2']

querydf = df_tmp.loc[df_tmp.blue_or_white == 'white']
querydf = querydf[~querydf['tokenized_requirement_2'].isnull()]
# querydf = df

keywords = list(itertools.chain(*querydf.loc[:,'tokenized_requirement_2']))
counts = Counter(keywords)

counts_df_ = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts_df_.columns = ['word','count']

counts_df_ = counts_df_.sort_values('count',ascending=False).reset_index(drop=True).iloc[:60,:]

In [88]:
counts_df_

,word,count
0,có,4655
1,các,3137
2,và,2853
3,kinh_nghiệm,2493
4,làm_việc,2093
5,năm,1770
6,tốt_nghiệp,1757
7,trong,1673
8,chuyên_ngành,1468
9,cơ_khí,1432


In [86]:
from operator import itemgetter 

'Trình độ: Từ trung cấp trở lên\n.Kinh nghiệm: Không yêu cầu kinh nghiệm (sẽ được đào taHam học hỏi, có mong muốn nâng cao kỹ năng, trình độ của bản thân.\nCó tư cách đạo đức tốt, làm việc nghiêm túc, trách nhiệm, chăm chỉ, có sức khỏe tốt\n.Biết vẽ AutoCad là một lợi thếCó kinh nghiệm, kỹ năng trong kỹ thuật, điện, điện tử, nước dân dụng, biết sử dụng các công cụ kỹ thuật điện và nước dân dụng cơ bản là một lợi thế'

In [113]:
df_tmp = df[df['industry'].str.contains('Cơ khí', regex=False)]

querydf = df_tmp.loc[df_tmp.blue_or_white == 'blue']
querydf = querydf[~querydf['tokenized_requirement_2'].isnull()]

keywords = list(itertools.chain(*querydf.loc[:,'tokenized_requirement_2']))

indexes = [i for i,x in enumerate(keywords) if x == 'thành_thạo']
indexes = [x+1 for x in indexes]


keywords2 = itemgetter(*indexes)(keywords)
counts = Counter(keywords2)
counts_df_ = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts_df_.columns = ['word','count']

counts_df_ = counts_df_.sort_values('count',ascending=False).reset_index(drop=True).iloc[:30,:]

In [114]:
counts_df_

,word,count
0,vi_tính,9
1,các,5
2,word,2
3,trong,1
4,trung_thực,1
5,lành_nghề,1
6,bản_vẽ,1
7,adobe,1
8,công_cụ,1
9,autocard,1


In [ ]:
date_object = datetime.date.today()
df.to_csv(f'aggregated-{date_object}.csv', index=False)

df.to_hdf(f'aggregated-{date_object}.h5', key='df', mode='w')

d = pd.read_hdf('data.h5', 'df')